In [2]:
#!pip install python-dotenv
#!pip install openai
#!pip install --upgrade langchain
#!pip install langchain-community
#!pip install langchain-openai
#!pip install langchain-core

In [3]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read loacl .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [4]:
# Account for deprecation of LLM model
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define the target date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 12, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-4o-mini"
else:
    llm_model = "gpt-3.5-turbo"

# Chat API: OpenAI
lets start a direct API call to openai

In [5]:
def get_completion(prompt, model=llm_model):
    # messages = [{'role':'user', 'content':prompt}]
    # response = openai.ChatCompletion.create(
    #     model = model,
    #     messages = messages,
    #     temperature = 0,
    # )

    response = client.responses.create(
        model= model,
        instructions="give direct answer.",
        input= prompt,
    )
    print(response.output_text)
    # return response.choices[0].message['content']

In [6]:
get_completion("What is 1+1?")

1 + 1 equals 2.


In [7]:
customer_email = """Your scientists were so preoccupied with whether or not they could, they didn't stop to think if they should.\
God creates dinosaurs. God destroys dinosaurs. God creates man. Man destroys God. Man creates dinosaurs.\
If there's one thing the history of evolution has taught us, it's that life will not be contained. Life breaks free.\
It expands into new territories. Painfully, maybe even dangerously. But, uh, there it is."""

In [8]:
style = """American english in calm and respectful tone"""

In [9]:
customer_email

"Your scientists were so preoccupied with whether or not they could, they didn't stop to think if they should.God creates dinosaurs. God destroys dinosaurs. God creates man. Man destroys God. Man creates dinosaurs.If there's one thing the history of evolution has taught us, it's that life will not be contained. Life breaks free.It expands into new territories. Painfully, maybe even dangerously. But, uh, there it is."

In [10]:
prompt = f"""Translate the text that is delimited by triple backticks into a style that is {style}.
text: ```{customer_email}```
"""
print(prompt)

Translate the text that is delimited by triple backticks into a style that is American english in calm and respectful tone.
text: ```Your scientists were so preoccupied with whether or not they could, they didn't stop to think if they should.God creates dinosaurs. God destroys dinosaurs. God creates man. Man destroys God. Man creates dinosaurs.If there's one thing the history of evolution has taught us, it's that life will not be contained. Life breaks free.It expands into new territories. Painfully, maybe even dangerously. But, uh, there it is.```



In [11]:
response = get_completion(prompt)

"Your scientists were so focused on whether they could do something, they didn't pause to consider whether they should. God creates dinosaurs. God destroys dinosaurs. God creates mankind. Mankind destroys God. Mankind creates dinosaurs. If there's one lesson from the history of evolution, it’s that life cannot be contained. Life breaks free. It ventures into new territories, often painfully and sometimes dangerously. But that’s the reality we face."


In [12]:
response

# Chat API : LangChain

In [13]:
from langchain_openai import ChatOpenAI

In [14]:
# To control the randomness and creativity of the generated text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0,model=llm_model)

In [15]:
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7aa320051df0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7aa320053980>, root_client=<openai.OpenAI object at 0x7aa3203ea270>, root_async_client=<openai.AsyncOpenAI object at 0x7aa320051e20>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

#### Prompt Template

In [16]:
template_string="""Translate the text that is delimited by triple backticks into a style that is {style}.\
text: ```{text}```
"""

In [17]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [18]:
prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}.text: ```{text}```\n'), additional_kwargs={})])

In [19]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [20]:
customer_messages = prompt_template.format_messages(style=style, text=customer_email)

In [21]:
print(type(customer_messages))

<class 'list'>


In [22]:
print(type(customer_messages[0]))

<class 'langchain_core.messages.human.HumanMessage'>


In [23]:
print(customer_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is American english in calm and respectful tone.text: ```Your scientists were so preoccupied with whether or not they could, they didn't stop to think if they should.God creates dinosaurs. God destroys dinosaurs. God creates man. Man destroys God. Man creates dinosaurs.If there's one thing the history of evolution has taught us, it's that life will not be contained. Life breaks free.It expands into new territories. Painfully, maybe even dangerously. But, uh, there it is.```



In [24]:
# Call LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)
print(customer_response.content)

Your scientists were so focused on whether they could do something that they didn't take the time to consider if they should. God creates dinosaurs. God destroys dinosaurs. God creates man. Man destroys God. Man creates dinosaurs. If there's one lesson we can learn from the history of evolution, it's that life cannot be contained. Life breaks free. It expands into new territories, sometimes painfully and even dangerously. But that is the reality we face.


#### Output Parsers
Let's start with defining how we would like the LLM output to look like:

In [25]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [26]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [27]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [28]:
messages = prompt_template.format_messages(text=customer_review)
response = chat.invoke(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there", "I think it's worth it for the extra features"]
}
```


### Parse the LLM output string into a Python dictionary

In [29]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [30]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [31]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [32]:
format_instructions = output_parser.get_format_instructions()

In [33]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [34]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt_template = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt_template.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [35]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [36]:
response = chat.invoke(messages)
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [37]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [38]:
type(output_dict)

dict

In [39]:
output_dict.get('delivery_days')

'2'